In [2]:
import cv2
import numpy as np
import glob
import scipy.ndimage as nd
import re
import os
from multiprocessing import Pool
import time


In [3]:
from os.path import isfile, join
import h5py
from skimage.util import view_as_blocks


def Down_Sample(image, block_size, func=np.sum, cval=0):

    if len(block_size) != image.ndim:
        raise ValueError("`block_size` must have the same length "
                         "as `image.shape`.")

    pad_width = []
    for i in range(len(block_size)):
        if block_size[i] < 1:
            raise ValueError("Down-sampling factors must be >= 1. Use "
                             "`skimage.transform.resize` to up-sample an "
                             "image.")
        if image.shape[i] % block_size[i] != 0:
            after_width = block_size[i] - (image.shape[i] % block_size[i])
        else:
            after_width = 0
        pad_width.append((0, after_width))

    image = np.pad(image, pad_width=pad_width, mode='constant',
                   constant_values=cval)

    out = view_as_blocks(image, block_size)

    for i in range(len(out.shape) // 2):
        out = func(out, axis=-1)

    return out

def RmIsolated_pixel(img):

    sat = np.pad(img, pad_width=1, mode='constant', constant_values=0)
    sat = np.cumsum(np.cumsum(sat, axis=0), axis=1)
    sat = np.pad(sat, ((1, 0), (1, 0)), mode='constant', constant_values=0)
    # These are all the possible overlapping 3x3 windows sums
    sum3x3 = sat[3:, 3:] + sat[:-3, :-3] - sat[3:, :-3] - sat[:-3, 3:]
    # This takes away the central pixel value
    sum3x3 -= img
    # This zeros all the isolated pixels
    img[sum3x3 == 0] = 0

    return img

In [4]:
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [5]:
def generic_laplace(input, derivative2, output=None, mode="reflect",
                    cval=0.0,
                    extra_arguments=(),
                    extra_keywords = None):
    """N-dimensional Laplace filter using a provided second derivative function
    Parameters
    ----------
    %(input)s
    derivative2 : callable
        Callable with the following signature::
            derivative2(input, axis, output, mode, cval,
                        *extra_arguments, **extra_keywords)
        See `extra_arguments`, `extra_keywords` below.
    %(output)s
    %(mode)s
    %(cval)s
    %(extra_keywords)s
    %(extra_arguments)s
    """
    if extra_keywords is None:
        extra_keywords = {}
    input = numpy.asarray(input)
    output, return_value = _ni_support._get_output(output, input)
    axes = list(range(input.ndim))
    if len(axes) > 0:
        derivative2(input, axes[0], output, mode, cval,
                    *extra_arguments, **extra_keywords)
        for ii in range(1, len(axes)):
            tmp = derivative2(input, axes[ii], output.dtype, mode, cval,
                              *extra_arguments, **extra_keywords)
            output += tmp
    else:
        output[...] = input[...]
    return return_value

def gaussian_laplace(input, sigma, output=None, mode="reflect",
                     cval=0.0, **kwargs):
    """Multidimensional Laplace filter using gaussian second derivatives.
    Parameters
    ----------
    %(input)s
    sigma : scalar or sequence of scalars
        The standard deviations of the Gaussian filter are given for
        each axis as a sequence, or as a single number, in which case
        it is equal for all axes.
    %(output)s
    %(mode)s
    %(cval)s
    Extra keyword arguments will be passed to gaussian_filter().
    """
    input = numpy.asarray(input)

    def derivative2(input, axis, output, mode, cval, sigma, **kwargs):
        order = [0] * input.ndim
        order[axis] = 2
        return gaussian_filter(input, sigma, order, output, mode, cval,
                               **kwargs)

    return generic_laplace(input, derivative2, output, mode, cval,
                           extra_arguments=(sigma,),
                           extra_keywords=kwargs)


def hist_match(source, template):
    """
    Adjust the pixel values of a grayscale image such that its histogram
    matches that of a target image

    Arguments:
    -----------
        source: np.ndarray
            Image to transform; the histogram is computed over the flattened
            array
        template: np.ndarray
            Template image; can have different dimensions to source
    Returns:
    -----------
        matched: np.ndarray
            The transformed output image
    """

    oldshape = source.shape
    source = source.ravel()
    template = template.ravel()

    # get the set of unique pixel values and their corresponding indices and
    # counts
    s_values, bin_idx, s_counts = np.unique(source, return_inverse=True,
                                            return_counts=True)
    t_values, t_counts = np.unique(template, return_counts=True)

    # take the cumsum of the counts and normalize by the number of pixels to
    # get the empirical cumulative distribution functions for the source and
    # template images (maps pixel value --> quantile)
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # interpolate linearly to find the pixel values in the template image
    # that correspond most closely to the quantiles in the source image
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    return interp_t_values[bin_idx].reshape(oldshape)

def hist_match_color(source,template):
    b_source=source[:,:,0]
    g_source=source[:,:,1]
    r_source=source[:,:,2]
    
    b_template=template[:,:,0]
    g_template=template[:,:,1]
    r_template=template[:,:,2]
    
    b_matched=hist_match(b_source,b_template)
    g_matched=hist_match(g_source,g_template)
    r_matched=hist_match(r_source,r_template)
    
    matched=cv2.merge([b_matched,g_matched,r_matched])
    return matched

In [6]:
def info(img8):
    print img8.dtype
    print img8.shape
    print type(img8)
    print np.max(img8)
    print np.min(img8)

In [7]:
def GetBlockList(overlap=10/2):
    x1=0
    y1=256
    blocklist=[]
    count = 0
    while (x1+256<10000):
        x2=0
        y2=256
        while (x2+256<12000):
            blocklist.append(np.asarray([x1,y1,x2,y2,count]))
            count = count + 1
            x2=y2-overlap
            y2=x2+512/2
        x1=y1-overlap
        y1=x1+512/2
    return blocklist

In [8]:
def GetNO(filename):
    return int(filename[-8:-4])

In [9]:
def blockprocess(img,metadata,filecount):
    #print type(metadata)
    sub=img[metadata[0]:metadata[1],metadata[2]:metadata[3]]
    #Gmask= np.max(sub,axis=2) != sub[:,:,1]
    #sub[Gmask]=0
    #sub[sub[:,:,1]<50] =0
    #sub=cv2.cvtColor(sub,cv2.COLOR_BGR2GRAY)
    cv2.imwrite('/scratch/PMD1475_SK/'+str(metadata[4])+'/'+str(filecount)+'.jp2',sub)

In [10]:
def OneFileprocess(filename):
    
    print 'processing '+filename
    start_time = time.time()
    
    img=cv2.imread(filename,-1)
    kk = np.min(img)
    img = img - kk
    #print info(img)
    img=cv2.medianBlur(img,3)
    LOG=nd.gaussian_laplace(img,3)
    #mediancp=np.copy(median)
    mask1 = LOG[:,:,0] + LOG[:,:,1] + LOG[:,:,2] < 1000      #background noise 40
    mask2 = img[:,:,1] < 500                             #not strong green (need to replace by better detect) 70
    mask3 = np.max(img[:,:,:],axis=2) == img[:,:,2]   #looks red(need to replace by autofluo detect)

    img[mask1 & mask2] = 0 
    img[mask3] = 0

    img=np.asarray(img,'uint8')
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=cv2.equalizeHist(img)

    img=Down_Sample(img,block_size=(2,2),func=np.max)
    img=np.asarray(img,'uint8')

    cv2.imwrite('/scratch/PMD1475_NOCUT/'+os.path.basename(filename),img)
    
    #print info(img)
    blocks=GetBlockList()
    for k in blocks:
        blockprocess(img,k,GetNO(filename))
        
    print ('Finishing ' + filename + ', took %s seconds' % (time.time() - start_time))

In [11]:
#main
inputfolder='/home/xli/toXuLi/PMD1475/PMD1475/*.jp2'
filelist=natural_sort(glob.glob(inputfolder))

In [22]:
img = cv2.imread(filelist[44],-1)

In [23]:
img.shape

(18000, 24000, 3)

In [27]:
OneFileprocess(filelist[44])

processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F17-2014.02.10-20.20.35_PMD1475_1_0049.jp2
True
Finishing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F17-2014.02.10-20.20.35_PMD1475_1_0049.jp2, took 317.343271971 seconds


In [12]:
p=Pool(10)
p.map(OneFileprocess,filelist)

processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F9-2014.02.10-17.51.33_PMD1475_1_0025.jp2
processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F14-2014.02.10-19.18.42_PMD1475_1_0040.jp2
processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F16-2014.02.10-20.00.17_PMD1475_2_0047.jp2
processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F21-2014.02.11-08.52.13_PMD1475_1_0061.jp2
processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F18-2014.02.10-20.41.55_PMD1475_3_0054.jp2
processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F2-2014.02.10-15.53.15_PMD1475_1_0004.jp2
processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F6-2014.02.10-17.00.14_PMD1475_3_0018.jp2
processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F11-2014.02.10-18.28.20_PMD1475_2_0032.jp2
processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F4-2014.02.10-16.26.31_PMD1475_2_0011.jp2
processing /home/xli/toXuLi/PMD1475/PMD1475/PMD1476&1475-F23-2014.02.11-09.35.38_PMD1475_2_0068.jp2
Fini

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [21]:
#post-process
GetBlockList()[0]

array([  0, 256,   0, 256,   0])

In [26]:
def FileProcess(i):
    print 'processing number '+str(i)
    #folder='/home/xli/M919_Green_SK/'+str(i)+'/'
    folder = '/scratch/PMD1475_SK/'+str(i)+'/'
    k=np.zeros((256,256,300))
    os.chdir(folder)
    kcount=0
    for file in glob.glob("*.jp2"):
        img=cv2.imread(folder+file,0)
        temp = np.zeros([256,256])
        temp[:img.shape[0],:img.shape[1]] = img
        
        k[:,:,kcount]=temp
        kcount=kcount+1
        
        
        thumbnail=np.max(k,axis=2)
        cv2.imwrite('/scratch/PMD1475_Thumbnail/'+str(i)+'.jpg',thumbnail)

In [27]:
data=range(1832)
s=Pool(32)
s.map(FileProcess,data)

processing number 15
processing number 30
processing number 0
processing number 105
processing number 165
processing number 75
processing number 45
processing number 150
processing number 360
processing number 210
processing number 195
processing number 225
processing number 300
processing number 60
processing number 240
processing number 285
processing number 255
processing number 180
processing number 465
processing number 450
processing number 330
processing number 375
processing number 345
processing number 315
processing number 120
processing number 270
processing number 420
processing number 435
processing number 135
processing number 390
processing number 90
processing number 405
processing number 376
processing number 196
processing number 166
processing number 151
processing number 391
processing number 436
processing number 46
processing number 91
processing number 286
processing number 226
processing number 406
processing number 361
processing number 181
processing number 24

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,